# Explore here

In [1]:
import pandas as pd

df = pd.read_excel("../datos_merged_1986_2023.xlsx")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11450 entries, 0 to 11449
Data columns (total 37 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   Unnamed: 0.1                11450 non-null  int64         
 1   Unnamed: 0                  11450 non-null  int64         
 2   track_id                    11450 non-null  object        
 3   track_name                  11450 non-null  object        
 4   popularity                  11450 non-null  int64         
 5   available_markets           11449 non-null  object        
 6   disc_number                 11450 non-null  int64         
 7   duration_ms                 11450 non-null  int64         
 8   explicit                    11450 non-null  bool          
 9   track_number                11450 non-null  int64         
 10  href                        11450 non-null  object        
 11  album_id                    11450 non-null  object    

In [3]:
df["year_s"] = df["year"].astype(str)
df["duration_ms_s"] = df["duration_ms"].astype(str)
df["popularity_s"] = df["popularity"].astype(str)


df["tags"]= df["track_name"] + " " + df["popularity_s"] + " " + df["duration_ms_s"] + " "  + df["year_s"] + " "  + df["artist_genres"]


df['tags'] = df['tags'].apply(lambda x: str(x).replace(";"," "))

In [4]:
df.iloc[0].tags

'True Colors 73 227600 1986 dance pop new wave pop permanent wave soft rock'

In [5]:
from sklearn.feature_extraction. text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(df["tags"])

model = NearestNeighbors(n_neighbors=6, algorithm="auto", metric="cosine")
model.fit(tfidf_matrix)

NearestNeighbors(metric='cosine', n_neighbors=6)

In [10]:
def lista_canciones (cancion):
    indice_cancion = df[df["track_name"] == cancion].index[0]
    distancia, indices = model.kneighbors(tfidf_matrix[indice_cancion])
    canciones_similares = [(df["track_name"] [i], distancia[0][j]) for j, i in enumerate(indices[0])]
    return canciones_similares[1:]

In [11]:
cancion_input = 'Raining Blood'
recomendaciones = lista_canciones(cancion_input)
print("Recomendaciones para '{}'".format(cancion_input))
for cancion, distancia in recomendaciones:
    print("- Canción: {}".format(cancion))


Recomendaciones para 'Raining Blood'
- Canción: Angel Of Death
- Canción: Postmortem
- Canción: South Of Heaven
- Canción: South Of Heaven
- Canción: War Ensemble


In [14]:
def str_canciones_recomendadas(cancion_input):
    recomendaciones = lista_canciones(cancion_input)
    resultado = "Recomendaciones para" + "<br />"

    for cancion, distancia in recomendaciones:
        resultado = resultado+"-Canción: "+cancion+"<br />"
        #print("- Canción: {}".format(cancion))
    return resultado

In [15]:
print(str_canciones_recomendadas("Talk Dirty To Me"))

Recomendaciones para<br />-Canción: Big Love - 2017 Remaster<br />-Canción: Shake Me<br />-Canción: Talk to Me<br />-Canción: Take Me Home Tonight<br />-Canción: Rock Me<br />


In [16]:
df.to_csv("datos_canciones_tags.csv")

In [ ]:
from pickle import dump

dump(model, open("nn_6_auto_cosine.model", "wb"))